In [26]:
import pandas as pd
import os

os.chdir(r'C:\Users\jf79\Repos\REAP-Analytics')
import myfuncs.myfuncs as mf

import warnings
warnings.filterwarnings('ignore')

path_to_data = r"C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\All Historical Data\Interactions Data"
os.chdir(path_to_data)
mf.read_directory()

master_data = pd.read_csv('Netcall Interactions Master Data.csv',low_memory=False)
master_data = mf.strip_dataframe(master_data)

Your Current Directory is: C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\All Historical Data\Interactions Data
DataFrame: files
Number of records: 15 | Number of fields: 1
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,File Names
0,id_check.csv
1,Interactions Apr 2025 (excl Cust Serv)
2,Interactions August 2025 (excl Cust Serv)
3,Interactions Feb 2023 - Apr 2024 (excl Cust Serv)
4,Interactions Feb 2025 (excl Cust Serv)
5,Interactions Jan 2022 - Dec 2022 (excl Cust Serv)
6,Interactions Jan 2024 - Jun 2024 (excl Cust Serv)
7,Interactions July 2025 (excl Cust Serv)
8,Interactions Jun 24 - Jan 25 (excl Cust Serv)
9,Interactions Mar 2025 (excl Cust Serv)


In [27]:
master_df = master_data.copy()

columns_to_drop = [
    'customername','assistedby',
    'casereference','ipaddress'
]
master_df.drop(columns=columns_to_drop, inplace=True)

master_df['nullcount'] = master_df.isna().sum(axis=1)
master_df.sort_values(by=['interactionid','nullcount'], inplace=True)
drop = master_df[master_df['interactionid'].duplicated(keep='first')]
master_df.drop(drop.index, inplace=True)
master_df.drop(columns='nullcount', inplace=True)

columns_to_change = ['timeended', 'timequeued']
for column in columns_to_change:
    master_df[column] = pd.to_datetime(master_df[column])

master_df['queuetime'] = master_df['timeended'] - master_df['timequeued']

columns_to_change = [
    'duration','talktime','holdtime',
    'wrapupduration','agenthandlingtime'
]
master_df[columns_to_change] = master_df[columns_to_change].apply(lambda x: x.str.replace(r'(?<!^)-', '', regex=True))
for column in columns_to_change:
    master_df[column] = pd.to_timedelta(master_df[column])

master_df['dateofqueue'] = master_df['timequeued'].dt.date

In [28]:
aggregated_interactions = master_df.groupby(['direction','type','dateofqueue','groupname','queue','result']).agg(
    number_of_interactions = ('interactionid', 'count'),
    wrap_upduration = ('wrapupduration','sum'),
    queue_time = ('queuetime','sum'),
    duration = ('duration','sum'),
    talk_time = ('talktime','sum'),
    hold_time = ('holdtime','sum'),
    agent_handling_time = ('agenthandlingtime','sum'),
).reset_index()

columns_to_change = [
    'wrap_upduration','queue_time','duration',
    'talk_time','hold_time','agent_handling_time'
]
for column in columns_to_change:
    aggregated_interactions[column].fillna(pd.Timedelta(0), inplace=True)
    aggregated_interactions[column] = aggregated_interactions[column].apply(lambda x: x.total_seconds())

mf.validate_data(aggregated_interactions)

#####################################################################################################################################################################
Validation Dataframe
Number of records: 229,876 | Number of fields: 13
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,direction,type,dateofqueue,groupname,queue,result,number_of_interactions,wrap_upduration,queue_time,duration,talk_time,hold_time,agent_handling_time
0,Inbound,Call,2022-01-04,ACE and Admissions,ACE and Admissions,Answered,15,4241.0,5580.0,5349.0,5349.0,0.0,9590.0
1,Inbound,Call,2022-01-04,Adult Education,General Enquiry,Abandoned,2,0.0,240.0,0.0,0.0,0.0,0.0
2,Inbound,Call,2022-01-04,Adult Education,General Enquiry,Answered,66,1954.0,25260.0,23670.0,23670.0,0.0,25624.0
3,Inbound,Call,2022-01-04,CAS Team 1 6945,CAS Team 1 6945 Queue,Answered,3,180.0,300.0,210.0,210.0,0.0,390.0
4,Inbound,Call,2022-01-04,CAS Team 2 6946,CAS Team 2 6946 Queue,Answered,3,180.0,360.0,295.0,295.0,0.0,475.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
229871,Outbound,Email,2025-09-02,Customer Services,Cost of Living Crisis Email,Sent,3,180.0,216.0,216.0,214.0,2.0,394.0
229872,Outbound,Email,2025-09-03,Customer Services,Cost of Living Crisis Email,Deleted,1,61.0,75.0,75.0,74.0,1.0,135.0
229873,Outbound,Email,2025-09-04,Customer Services,Cost of Living Crisis Email,Sent,2,134.0,139.0,138.0,137.0,1.0,271.0
229874,Outbound,Email,2025-09-05,Customer Services,Cost of Living Crisis Email,Sent,2,82.0,85.0,83.0,83.0,0.0,165.0


Unique Values
DataFrame: unique_df


,direction,type,dateofqueue,groupname,queue,result,number_of_interactions,wrap_upduration,queue_time,duration,talk_time,hold_time,agent_handling_time
0,Inbound,Call,2022-01-01,ACE and Admissions,ACE and Admissions,Abandoned,1.0,0.0,-32.0,0.0,-98142.0,0.0,0.0
1,Outbound,Email,2022-01-02,ASB UNIT,ASB UNIT,Answered,2.0,1.0,-8.0,1.0,-90007.0,1.0,1.0
2,--,Messaging,2022-01-03,Accessible Transport,Accessible Transport,Callback,3.0,2.0,-7.0,2.0,-84754.0,2.0,2.0
3,--,--,2022-01-04,Adult Education,Accessible Transport Blue Badge,Deleted,4.0,3.0,-6.0,3.0,0.0,3.0,3.0
4,--,--,2022-01-05,Adult Social Care,Accessible Transport Callback,Discarded,5.0,4.0,-5.0,4.0,1.0,4.0,4.0
5,--,--,2022-01-06,CAS Team 1 6945,Accessible Transport Email,Failed,6.0,5.0,-4.0,5.0,2.0,5.0,5.0
6,--,--,2022-01-07,CAS Team 2 6946,Accessible Transport Freedom Passes,Other,7.0,6.0,-3.0,6.0,3.0,6.0,6.0
7,--,--,2022-01-08,CAS Team 3 6947,Accessible Transport Taxi Cards,Pending,8.0,7.0,-2.0,7.0,4.0,7.0,7.0
8,--,--,2022-01-09,CAS Team 4 6948,Adult Social Care,Redirected,9.0,8.0,-1.0,8.0,5.0,8.0,8.0
9,--,--,2022-01-10,CSD Reablement 6613,Agent Transfers,Replied,10.0,9.0,0.0,9.0,6.0,9.0,9.0



DataFrame: info_df
Max number of non-null records: 229,876


,Field Name,No. of Unique Values,No. of Null Values,No. of Blank Values,Data Type
0,direction,2,--,--,object
1,type,3,--,--,object
2,dateofqueue,"1,337",--,--,object
3,groupname,51,--,--,object
4,queue,202,--,--,object
5,result,12,--,--,object
6,number_of_interactions,431,--,--,int64
7,wrap_upduration,"10,204",--,--,float64
8,queue_time,"46,477",--,--,float64
9,duration,"28,499",--,--,float64



Summary statistics:


,number_of_interactions,wrap_upduration,queue_time,duration,talk_time,hold_time,agent_handling_time
count,229876.000000,2.298760e+05,2.298760e+05,2.298760e+05,229876.000000,2.298760e+05,2.298760e+05
mean,12.120704,8.019312e+02,3.229936e+05,9.963250e+03,3090.739346,6.872551e+03,4.486009e+03
std,32.353383,8.838439e+03,3.986786e+06,4.728142e+05,8709.339363,4.723372e+05,1.779697e+04
min,1.000000,0.000000e+00,-3.200000e+01,0.000000e+00,-98142.000000,0.000000e+00,0.000000e+00
25%,1.000000,0.000000e+00,2.750000e+02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
50%,3.000000,3.100000e+01,1.290000e+03,3.940000e+02,332.000000,0.000000e+00,4.840000e+02
75%,11.000000,2.930000e+02,6.790250e+03,2.447000e+03,2044.000000,1.050000e+02,2.702000e+03
max,4154.000000,2.313121e+06,6.078473e+08,2.213014e+08,313159.000000,2.212321e+08,2.334216e+06


End of data validation
#####################################################################################################################################################################


In [29]:
mf.export_file(
    aggregated_interactions,
    directory = path_to_data,
    df_name='Netcall Dashboard Data',
    file_type='csv'
)

Exporting Netcall Dashboard Data to CSV...
@ C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\All Historical Data\Interactions Data\Netcall Dashboard Data.csv

Successfully exported Netcall Dashboard Data to CSV
